# 读取数据

In [97]:
import pandas as pd
import jieba

In [98]:
df_news = pd.read_csv('./train2.csv')
df_news.head()

,title,label
0,持续追踪丨俄再度炮击赫尔松，泽连斯基呼吁西方加强乌防空,国际
1,COP15专访｜穿汉服的中国代表团成员讲述谈判背后的故事,国际
2,泰国一艘军舰沉没，搜救队正在寻找33名失踪船员,国际
3,圆桌｜威斯特伐利亚体系从未退场？俄乌冲突下的欧洲之变,国际
4,法国败北马克龙表态：决赛不是提前写好的剧本，希望德尚留任,国际


In [99]:
content = df_news.title.values.tolist()
print (content[0])

持续追踪丨俄再度炮击赫尔松，泽连斯基呼吁西方加强乌防空


# 对数据进行结巴分词

In [100]:
# 训练集使用结巴分词
content_S = []
for line in content:
    current_segment = jieba.lcut(line)
    if len(current_segment) > 1 and current_segment != '\r\n': #换行符
        content_S.append(current_segment)
print(content_S[1000]) #(list of list)


['夜读', '｜', '照亮', '梅西', '的', '光', '，', '究竟', '来自', '何处', '？']


In [101]:
# 格式转化
df_content = pd.DataFrame({'content_S':content_S})
print(df_content.head())

                                           content_S
0  [持续, 追踪, 丨, 俄, 再度, 炮击, 赫尔松, ，, 泽, 连斯基, 呼吁, 西方,...
1  [COP15, 专访, ｜, 穿, 汉服, 的, 中国, 代表团, 成员, 讲述, 谈判, ...
2  [泰国, 一艘, 军舰, 沉没, ，, 搜救, 队, 正在, 寻找, 33, 名, 失踪, 船员]
3  [圆桌, ｜, 威, 斯特伐, 利亚, 体系, 从未, 退场, ？, 俄乌, 冲突, 下, ...
4  [法国, 败北, 马克, 龙, 表态, ：, 决赛, 不是, 提前, 写, 好, 的, 剧本...


# 去除数据停用词

In [102]:
#读取查看停用词表

stopwords = pd.read_csv("./hit_stopwords.txt",index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8') #list
stopwords.head()

,stopword
0,丨
1,｜
2,｜
3,———
4,》），


In [103]:
# 对训练集进行停用词处理

# 停用词函数
def drop_stopwords(contents, stopwords):
    contents_clean = []
    all_words = []
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
            all_words.append(str(word))
        contents_clean.append(line_clean)
    return contents_clean, all_words
    # print (contents_clean)

In [104]:
### 对训练集进行停用词转化 

# 对训练集的值进行转化
contents = df_content.content_S.values.tolist()

# 对停用词进行转化
stopwords = stopwords.stopword.values.tolist()

# 使用停用词
contents_clean, all_words = drop_stopwords(contents, stopwords) 

# 每一列的分词
df_content = pd.DataFrame({'contents_clean':contents_clean})
print(df_content.head())

print()

# 所有词语
df_all_words = pd.DataFrame({'all_words':all_words})
print(df_all_words.head())

###转化结束

                                      contents_clean
0  [持续, 追踪, 俄, 再度, 炮击, 赫尔松, 泽, 连斯基, 呼吁, 西方, 加强, 乌...
1    [COP15, 专访, 穿, 汉服, 中国, 代表团, 成员, 讲述, 谈判, 背后, 故事]
2     [泰国, 一艘, 军舰, 沉没, 搜救, 队, 正在, 寻找, 33, 名, 失踪, 船员]
3    [圆桌, 威, 斯特伐, 利亚, 体系, 从未, 退场, 俄乌, 冲突, 下, 欧洲, 之变]
4  [法国, 败北, 马克, 龙, 表态, 决赛, 不是, 提前, 写, 好, 剧本, 希望, ...

  all_words
0        持续
1        追踪
2         俄
3        再度
4        炮击


In [105]:
df_news

,title,label
0,持续追踪丨俄再度炮击赫尔松，泽连斯基呼吁西方加强乌防空,国际
1,COP15专访｜穿汉服的中国代表团成员讲述谈判背后的故事,国际
2,泰国一艘军舰沉没，搜救队正在寻找33名失踪船员,国际
3,圆桌｜威斯特伐利亚体系从未退场？俄乌冲突下的欧洲之变,国际
4,法国败北马克龙表态：决赛不是提前写好的剧本，希望德尚留任,国际
...,...,...
4995,北京第四批次6宗住宅用地收金135亿元，“热度达到了较高水平”地产界,财经
4996,杭州第四批次供地：8宗宅地收金98.9亿元，其中3宗达到上限价格地产界,财经
4997,独家｜知情人谈张核子和核酸生意：深圳坐拥带停机坪别墅，张姗姗是谁？10%公司,财经
4998,放量大涨沪指收复60日均线，反弹持续性如何？或挑战半年线牛市点线面,财经


In [106]:
# 提取关键词
import jieba.analyse
index = 0


print (df_news['title'][index])
content_S_str = "".join(content_S[index])
print ("  ".join(jieba.analyse.extract_tags(content_S_str, topK=5, withWeight=False)))#3输出前五个关键词

持续追踪丨俄再度炮击赫尔松，泽连斯基呼吁西方加强乌防空
赫尔松  连斯基  炮击  追踪  呼吁


# LDA模型建立

In [107]:
# LDA模型建立

## 训练集
from gensim import corpora, models, similarities
import gensim
#做映射，相当于词袋
dictionary = corpora.Dictionary(contents_clean) ##格式要求：list of list形式，分词好的的整个语料
corpus = [dictionary.doc2bow(sentence) for sentence in contents_clean]  #语料
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20) #类似Kmeans自己指定K值
print (lda.print_topic(1, topn=5)) ##第一个主题，关键词5个

0.023*"我国" + 0.023*"汽车" + 0.020*"价格" + 0.012*"中" + 0.010*"出售"


In [108]:
# 输出20个主题的的关键词和权重

## 训练集
for topic in lda.print_topics(num_topics=20, num_words=5):
    print (topic[1])

0.022*"健康" + 0.020*"公司" + 0.019*"中国" + 0.014*"个人" + 0.011*"防疫"
0.023*"我国" + 0.023*"汽车" + 0.020*"价格" + 0.012*"中" + 0.010*"出售"
0.045*"11" + 0.036*"月" + 0.025*"同比" + 0.024*"专家" + 0.020*"银行"
0.049*"公司" + 0.022*"经济" + 0.020*"集团" + 0.019*"中央" + 0.015*"元"
0.020*"三大" + 0.019*"发展" + 0.016*"服务" + 0.015*"数据" + 0.013*"50"
0.029*"资金" + 0.022*"公司" + 0.016*"增" + 0.015*"出海" + 0.012*"措施"
0.038*"增长" + 0.018*"科学" + 0.017*"加快" + 0.015*"收" + 0.014*"月"
0.040*"企业" + 0.015*"部门" + 0.013*"逾" + 0.011*"两年" + 0.011*"20"
0.044*"月" + 0.025*"12" + 0.021*"多地" + 0.017*"金融" + 0.017*"11"
0.045*"消费" + 0.018*"2023" + 0.014*"违规" + 0.013*"年" + 0.012*"铁路"
0.021*"中国" + 0.020*"预期" + 0.019*"约" + 0.018*"新冠" + 0.015*"病毒"
0.023*"解读" + 0.022*"拟" + 0.019*"经济" + 0.019*"用于" + 0.018*"新"
0.141*"公司" + 0.016*"月" + 0.014*"明年" + 0.014*"监管" + 0.013*"11"
0.042*"工作" + 0.026*"人民币" + 0.023*"经济" + 0.022*"会议" + 0.015*"涨幅"
0.021*"恢复" + 0.015*"新冠" + 0.012*"5" + 0.012*"逾" + 0.010*"8"
0.046*"市场" + 0.024*"科学" + 0.016*"银行" + 0.014*"首个" + 0.012*"项目"
0.020

In [109]:
# 数据转化

## 训练集
df_train = pd.DataFrame({'contents_clean':contents_clean,'label':df_news['label']})
print(df_train.tail())




## 查看训练集的数据和不同的label
print(df_train.tail())
print(df_train.label.unique())

                                         contents_clean label
4995  [北京, 第四, 批次, 6, 宗, 住宅, 用地, 收金, 135, 亿元, 热度, 达到...    财经
4996  [杭州, 第四, 批次, 供地, 8, 宗, 宅地, 收金, 98.9, 亿元, 3, 宗,...    财经
4997  [独家, 知情人, 谈张, 核子, 核酸, 生意, 深圳, 坐, 拥带, 停机坪, 别墅, ...    财经
4998  [放量, 大涨, 沪, 指, 收复, 60, 日, 均线, 反弹, 持续性, 挑战, 半年线...    财经
4999  [三天, 涨, 超, 30%, 翰宇, 药业, 预防, 新冠, 多肽, 鼻, 喷, 已, 紧...    财经
                                         contents_clean label
4995  [北京, 第四, 批次, 6, 宗, 住宅, 用地, 收金, 135, 亿元, 热度, 达到...    财经
4996  [杭州, 第四, 批次, 供地, 8, 宗, 宅地, 收金, 98.9, 亿元, 3, 宗,...    财经
4997  [独家, 知情人, 谈张, 核子, 核酸, 生意, 深圳, 坐, 拥带, 停机坪, 别墅, ...    财经
4998  [放量, 大涨, 沪, 指, 收复, 60, 日, 均线, 反弹, 持续性, 挑战, 半年线...    财经
4999  [三天, 涨, 超, 30%, 翰宇, 药业, 预防, 新冠, 多肽, 鼻, 喷, 已, 紧...    财经
['国际' '思想' '时事' '科技' '财经']


In [110]:
# 对数据标签进行map映射
label_mapping = {"国际": 0, "思想": 1, "时事": 2, "科技": 3, "财经":4}
df_train['label'] = df_train['label'].map(label_mapping) ##变换label

df_train.head()

,contents_clean,label
0,"[持续, 追踪, 俄, 再度, 炮击, 赫尔松, 泽, 连斯基, 呼吁, 西方, 加强, 乌...",0
1,"[COP15, 专访, 穿, 汉服, 中国, 代表团, 成员, 讲述, 谈判, 背后, 故事]",0
2,"[泰国, 一艘, 军舰, 沉没, 搜救, 队, 正在, 寻找, 33, 名, 失踪, 船员]",0
3,"[圆桌, 威, 斯特伐, 利亚, 体系, 从未, 退场, 俄乌, 冲突, 下, 欧洲, 之变]",0
4,"[法国, 败北, 马克, 龙, 表态, 决赛, 不是, 提前, 写, 好, 剧本, 希望, ...",0


# 贝叶斯模型建立

In [111]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(df_train['contents_clean'].values, df_train['label'].values, test_size=0.2, random_state=1, shuffle=True,stratify=df_train['label'].values)

words = []
for line_index in range(len(x_train)):  
    try:
        #x_train[line_index][word_index] = str(x_train[line_index][word_index])
        words.append(' '.join(x_train[line_index]))
    except:
        print (line_index)
     
     
test_words = []
for line_index in range(len(x_test)):
    try:
        #x_train[line_index][word_index] = str(x_train[line_index][word_index])
        test_words.append(' '.join(x_test[line_index]))
    except:
         print (line_index)
         


In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF
from sklearn.naive_bayes import MultinomialNB
from sklearn.calibration import CalibratedClassifierCV

tdidf = TFIDF()

tdidf = tdidf.fit(words)
x_train = tdidf.transform(words).toarray()
x_test = tdidf.transform(test_words).toarray()

model = CalibratedClassifierCV(MultinomialNB(), cv=2, method='isotonic')
model = model.fit(x_train, y_train)

y_pred = model.predict(x_test)
proba = model.predict_proba(x_test)

score = model.score(x_test, y_test)
score

0.855

In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF
from sklearn.naive_bayes import MultinomialNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import  SVC
from sklearn.metrics import accuracy_score as acc

tdidf = TFIDF()

tdidf = tdidf.fit(words)
x_train = tdidf.transform(words)
x_test = tdidf.transform(test_words)

# model = CalibratedClassifierCV(MultinomialNB(), cv=2, method='isotonic')
model = SVC(C=0.8)

model.fit(x_train,y_train)
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("训练集的准确度:",acc(y_train,train_pred))
print("测试集的准确度:",acc(y_test,test_pred))

训练集的准确度: 0.9965
测试集的准确度: 0.873


In [114]:
model = CalibratedClassifierCV(SVC(C=0.8), cv=5, method='isotonic')
model.fit(x_train,y_train)
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("训练集的准确度:",acc(y_train,train_pred))
print("测试集的准确度:",acc(y_test,test_pred))

训练集的准确度: 0.99725
测试集的准确度: 0.875
